#### 다음 실습 코드는 학습 목적으로만 사용 바랍니다. 문의 : audit@korea.ac.kr 임성열 Ph.D.

In [1]:
%pip install transformers torch

  Using cached safetensors-0.5.3-cp38-abi3-macosx_11_0_arm64.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 24.1 MB/s eta 0:00:00a 0:00:01
Using cached safetensors-0.5.3-cp38-abi3-macosx_11_0_arm64.whl (418 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [transformers] [transformers]
Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# 모델과 토크나이저 준비 (가볍게 'gpt2' 기본 모델 사용)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# 문장 생성 함수
def generate_text(prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    outputs = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id  # GPT2는 pad_token이 없으므로 eos_token으로 대체
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# 생성 실습
prompt = "The capital of South Korea is"
print(generate_text(prompt, 50))


# 이 코드를 실행하면 "Seoul" 이라고 정답을 줄 수도 있지만, 가끔 엉뚱한 내용으로 답하는 경우도 있다. (= Hallucination)

/Users/phoenix/Eagle/2025_교육-KDT/LLM 실습/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The capital of South Korea is a bastion of its prosperity, and its economy is growing at an even faster rate than most of the world's.

South Korea has the world's largest economy at over $100 trillion, and it is the


In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# 모델과 토크나이저 준비
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# 문장 생성 함수
def generate_text(prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id  # GPT-2는 pad_token_id가 없어 eos_token_id로 대체
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# 가상 지식베이스 입력 (미니 사전)
knowledge_base = {
    "South Korea": "The capital of South Korea is Seoul.",
    "France": "The capital of France is Paris.",
    "Italy": "The capital of Italy is Rome."
}

# 검색 함수
def retrieve_fact(query):
    for key in knowledge_base:
        if key.lower() in query.lower():
            return knowledge_base[key]
    return "I don't know."

# 생성 함수 + 지식 참조
def generate_with_retrieval(query):
    fact = retrieve_fact(query)
    prompt = fact + " " + query
    return generate_text(prompt, 50)

# RAG 스타일 생성 실습
query = "What is the capital of South Korea?"
print(generate_with_retrieval(query))


# 이렇게 하면 모델이 문장을 지어내기 전에, 검색된 진짜 지식(fact)을 넣어주기 때문에 환각이 줄어든다.

The capital of South Korea is Seoul. What is the capital of South Korea?

The capital city of Seoul is located in the South. What is the capital of South Korea?

South Korea is home to a large number of different cultural
